# Import libraries and metrics

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict as dd

RESULTS_PATH = "results/"
CLIENTS_METRICS = "clients.json"
NODES_METRICS = "node%d.json"

PLOTS_OUT_PATH = "plots/"

with open(RESULTS_PATH + CLIENTS_METRICS, "r") as f:
    file = open(RESULTS_PATH + CLIENTS_METRICS)
    clients_metrics = list(map(lambda x: json.loads(x), file.readlines()))

nodes_metrics = {}
for i in range(1, 5):
    with open(RESULTS_PATH + NODES_METRICS % i, "r") as f:
        file = open(RESULTS_PATH + NODES_METRICS % i)
        nodes_metrics[i] = list(map(lambda x: json.loads(x), file.readlines()))


# Results

## Correctness Tests

### Correct Blockchain Order

In [ ]:
# Check if 2f+1 nodes have the same number of blocks
num_blocks = {}
for i in range(1, 5):
    num_blocks[i] = len(list(filter(lambda x: x["metric"] == "committed_block", nodes_metrics[i])))
if len(set(num_blocks.values())) > 1:
    print("Different number of blocks in nodes")
    print(num_blocks)
else:
    print("All nodes have the same number of blocks: " + str(num_blocks[1]))

# Check if all nodes have the same blocks in same order
blocks = dd(dict)
for i in range(1, 5):
    for block in list(filter(lambda x: x["metric"] == "committed_block", nodes_metrics[i])):
        blocks[i][int(block["seq"])] = block["hash"]

for i in range(1, 5):
    for j in range(1, 5):
        for k in range(1, np.max(list(num_blocks.values())) + 1):
            try:
                if blocks[i][k] != blocks[j][k] and blocks[i][k]:
                    print("ERROR: Different blocks in nodes in seq=" + str(k))
                    print("Node " + str(i) + ": " + blocks[i][k])
                    print("Node " + str(j) + ": " + blocks[j][k])
                    pass
            except KeyError:
                continue

print("All nodes have the same blocks in same order")

## Simple Statistics

### Average Block Size

In [ ]:
avg_block_size = np.mean(list(map(lambda x: int(x["num_ops"]), filter(lambda x: x["metric"] == "committed_block", nodes_metrics[1]))))
print("Average block size: " + str(avg_block_size) + " operations")

### Average immediate reply time

In [ ]:
avg_reply = np.mean(list(map(lambda x: int(x["latency"]), filter(lambda x: x["metric"] == "operation_reply", clients_metrics))))
print("Average reply time: " + str(avg_reply) + " ms")

### Average operation execution time

In [ ]:
avg_exec = np.mean(list(map(lambda x: int(x["latency"]), filter(lambda x: x["metric"] == "operation_executed", clients_metrics))))
print("Average execution time: " + str(avg_exec) + " ms")

### Executed Operations Throughput

In [ ]:
# calculate start and end
elapsed = int(clients_metrics[-1]["time"]) - int(clients_metrics[0]["time"])

# calculate number of blocks
num_blocks = len(list(filter(lambda x: x["metric"] == "operation_executed", clients_metrics)))

# calculate throughput
throughput = num_blocks / (elapsed / 1000)

print("Throughput: " + str(throughput) + " ops/s")

### Block Throughput

In [ ]:
# calculate start and end
elapsed = int(nodes_metrics[3][-1]["time"]) - int(nodes_metrics[3][0]["time"])

# calculate number of blocks
num_blocks = len(list(filter(lambda x: x["metric"] == "committed_block", nodes_metrics[2])))

# calculate throughput
throughput = num_blocks / (elapsed / 1000)

print("Throughput: " + str(throughput) + " blocks/s")


## Plotting

### Throughput-Latency